In [1]:
%pip install --upgrade pip
%pip install --disable-pip-version-check \
    torch==1.13.1 \
    torchdata==0.5.1 --quiet
%pip install datasets
%pip install \
    transformers==4.27.2 \
    datasets==2.11.0 \
    evaluate==0.4.0 \
    rouge_score==0.1.2 \
    loralib==0.1.1 \
    peft==0.3.0 --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 20.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.2.1+cu121 requires torch==2.2.1, but you have torch 1.13.1 which is incompatible.
torchtext 0.17.1 requires torch

In [2]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np

In [3]:
huggingface_dataset_name = "d0rj/wikisum"

dataset = load_dataset(huggingface_dataset_name)

dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/35775 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/d0rj___parquet/d0rj--wikisum-01f0daf35fd9bb6a/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    validation: Dataset({
        features: ['url', 'title', 'summary', 'article', 'step_headers'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['url', 'title', 'summary', 'article', 'step_headers'],
        num_rows: 2000
    })
    train: Dataset({
        features: ['url', 'title', 'summary', 'article', 'step_headers'],
        num_rows: 35775
    })
})

In [4]:
model_name='google/flan-t5-base'

original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [5]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(original_model))

trainable model parameters: 247577856
all model parameters: 247577856
percentage of trainable model parameters: 100.00%


In [10]:
#Test the Model with Zero Shot Inferencing
index = 200

dialogue = dataset['test'][index]['article']
summary = dataset['test'][index]['summary']

prompt = f"""
Summarize the following conversation.

{dialogue}

Summary:
"""

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the appropriate device
original_model.to(device)

inputs = tokenizer(prompt, return_tensors='pt')
# Move the input tensors to the appropriate device
inputs = {key: value.to(device) for key, value in inputs.items()}

# Generate output
output = tokenizer.decode(
    original_model.generate(
        inputs["input_ids"],
        max_new_tokens=200,
    )[0].cpu(),  # Move the output back to CPU before decoding
    skip_special_tokens=True
)



# output = tokenizer.decode(
#     original_model.generate(
#         inputs["input_ids"],
#         max_new_tokens=200,
#     )[0],
#     skip_special_tokens=True
# )

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

Token indices sequence length is longer than the specified maximum sequence length for this model (913 > 512). Running this sequence through the model will result in indexing errors


---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Summarize the following conversation.

Hold the teeth on one side of the key over an open flame until they are charred. Use a lighter or a lit candle. Move the teeth of the key back and forth over the flame. It should take around 1 or 2 minutes to blacken one side of the key. Once the key is charred on one side, set it down until it is cool to the touch. Use a pair of pliers to hold the key so you don't burn your fingers. Stick a piece of clear tape on the blackened side of the key. Cut a piece of clear tape the same length as the key. Standard sized clear tape should work for most house keys. Carefully lay it on top of the key so it doesn't crease. Press the tape firmly onto the blackened side of the key so the char transfers. The outline of the key will transfer to the sticky side of the tape. Leave a fingertip-sized bit of tape at the end of the key so it's easier to re

In [11]:
#parameter Efficient Fine-Tuning (PEFT)
#Setup the PEFT/LoRA model for Fine-Tuning
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r=32, # Rank
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM # FLAN-T5
)



In [16]:
#Add LoRA adapter layers/parameters to the original LLM to be trained.
peft_model = get_peft_model(original_model,
                            lora_config)
print(print_number_of_trainable_model_parameters(peft_model))

trainable model parameters: 3538944
all model parameters: 251116800
percentage of trainable model parameters: 1.41%


In [12]:
def tokenize_function(example):
    start_prompt = 'Summarize the following conversation.\n\n'
    end_prompt = '\n\nSummary: '
    prompt = [start_prompt + dialogue + end_prompt for dialogue in example["article"]]
    example['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
    example['labels'] = tokenizer(example["summary"], padding="max_length", truncation=True, return_tensors="pt").input_ids

    return example

# The dataset actually contains 3 diff splits: train, validation, test.
# The tokenize_function code is handling all data across all splits in batches.
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['url', 'title', 'summary', 'article', 'step_headers'])



Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/35775 [00:00<?, ? examples/s]

In [13]:
tokenized_datasets = tokenized_datasets.filter(lambda example, index: index % 100 == 0, with_indices=True)


Filter:   0%|          | 0/2000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/2000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/35775 [00:00<?, ? examples/s]

In [14]:
print(f"Shapes of the datasets:")
print(f"Training: {tokenized_datasets['train'].shape}")
print(f"Validation: {tokenized_datasets['validation'].shape}")
print(f"Test: {tokenized_datasets['test'].shape}")

print(tokenized_datasets)

Shapes of the datasets:
Training: (358, 2)
Validation: (20, 2)
Test: (20, 2)
DatasetDict({
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 20
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 20
    })
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 358
    })
})


In [17]:
output_dir = f'./peft-WikiSum-summary-training-{str(int(time.time()))}'

# Define the fraction of the dataset you want to use (e.g., 10%)
subset_fraction = 0.1

# Calculate the number of samples to use
subset_size = int(len(dataset["train"]) * subset_fraction)

# Create a subset of the dataset
subset_dataset = dataset["train"].shuffle(seed=42).select(range(subset_size))

# Tokenize the subset
tokenized_subset = subset_dataset.map(tokenize_function, batched=True)


print(tokenized_subset)
# Define training arguments and create Trainer instance.

peft_training_args = TrainingArguments(
    output_dir=output_dir,
    auto_find_batch_size=4,
    learning_rate=1e-3, # Higher learning rate than full fine-tuning.
    num_train_epochs=10,
    logging_steps=1,
    max_steps=10
)

peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=tokenized_subset,
)


Dataset({
    features: ['url', 'title', 'summary', 'article', 'step_headers', 'input_ids', 'labels'],
    num_rows: 3577
})


In [18]:
peft_trainer.train()

peft_model_path="./peft-Wikisum-summary-checkpoint-local"

peft_trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
1,40.250000
2,37.500000
3,36.500000
4,34.500000
5,33.250000
6,32.000000
7,28.500000
8,27.375000
9,27.500000
10,27.625000


('./peft-Wikisum-summary-checkpoint-local/tokenizer_config.json',
 './peft-Wikisum-summary-checkpoint-local/special_tokens_map.json',
 './peft-Wikisum-summary-checkpoint-local/spiece.model',
 './peft-Wikisum-summary-checkpoint-local/added_tokens.json',
 './peft-Wikisum-summary-checkpoint-local/tokenizer.json')

In [19]:
from peft import PeftModel, PeftConfig

peft_model_base = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base", torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")

peft_model = PeftModel.from_pretrained(peft_model_base,
                                       './peft-Wikisum-summary-checkpoint-local',
                                       torch_dtype=torch.bfloat16,
                                       is_trainable=False)

In [20]:
#The number of trainable parameters will be 0 due to is_trainable=False setting:
print(print_number_of_trainable_model_parameters(peft_model))

trainable model parameters: 0
all model parameters: 251116800
percentage of trainable model parameters: 0.00%


In [21]:
index = 200
dialogue = dataset['test'][index]['article']
human_baseline_summary = dataset['test'][index]['summary']

prompt = f"""
Summarize the following conversation.

{dialogue}

Summary:
"""

# Tokenize the input prompt and make sure it's on the same device as the model
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(original_model.device)

original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{human_baseline_summary}')
print(dash_line)
print(f'ORIGINAL MODEL:\n{original_model_text_output}')


Token indices sequence length is longer than the specified maximum sequence length for this model (913 > 512). Running this sequence through the model will result in indexing errors


---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
You can have a copy of your key made by visiting a hardware store, car dealership, or key kiosk. Take your key with you to a hardware store and choose a blank replacement that matches the thickness of your original key. Ask the workers to make a copy for you, which takes a few minutes to complete. If you have a car key with a transponder chip, you'll need to go to your car's dealership. You can also use a key-cutting kiosk to make a copy of common keys. Take your key to one of them and follow the onscreen directions to have a copy made.
---------------------------------------------------------------------------------------------------
ORIGINAL MODEL:
Burn the key. Glue the key to the key. Place a piece of tape on the key. Tape the key onto the key. Place the tape on a credit card. Cut out the charred side of the key. Place the tape on a card. Place the key on the 

In [22]:
index = 200
human_baseline_summary = dataset['test'][index]['summary']
dialogue = dataset['test'][index]['article']

prompt = f"""
Summarize the following conversation.

{dialogue}

Summary: """

# Tokenize the input prompt and make sure it's on the same device as the models
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(original_model.device)

# Move the PEFT model to the same device as the original model
peft_model = peft_model.to(original_model.device)

# Generate summaries using the original model and PEFT model
with torch.no_grad():
    original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
    peft_model_outputs = peft_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))

original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)
peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{human_baseline_summary}')
print(dash_line)
print(f'ORIGINAL MODEL:\n{original_model_text_output}')
print(dash_line)
print(f'PEFT MODEL: {peft_model_text_output}')


---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
You can have a copy of your key made by visiting a hardware store, car dealership, or key kiosk. Take your key with you to a hardware store and choose a blank replacement that matches the thickness of your original key. Ask the workers to make a copy for you, which takes a few minutes to complete. If you have a car key with a transponder chip, you'll need to go to your car's dealership. You can also use a key-cutting kiosk to make a copy of common keys. Take your key to one of them and follow the onscreen directions to have a copy made.
---------------------------------------------------------------------------------------------------
ORIGINAL MODEL:
Immediately char the key. Place a piece of clear tape on the key. Place the tape on the key. Remove the tape. Remove the tape. Place the key on a card. Cut the key. Turn the key.
--------------------------------------

In [23]:
rouge = evaluate.load('rouge')


In [24]:
dialogues = dataset['test'][0:10]['article']
human_baseline_summaries = dataset['test'][0:10]['summary']

original_model_summaries = []
peft_model_summaries = []

# Move both models to the same device (CPU or GPU)
device = original_model.device  # Assuming original_model and peft_model are on the same device
original_model = original_model.to(device)
peft_model = peft_model.to(device)

for idx, dialogue in enumerate(dialogues):
    prompt = f"""
Summarize the following conversation.

{dialogue}

Summary: """

    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

    human_baseline_text_output = human_baseline_summaries[idx]

    with torch.no_grad():
        original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
        original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

        peft_model_outputs = peft_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
        peft_model_text_output = tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True)

    original_model_summaries.append(original_model_text_output)
    peft_model_summaries.append(peft_model_text_output)

zipped_summaries = list(zip(human_baseline_summaries, original_model_summaries, peft_model_summaries))

df = pd.DataFrame(zipped_summaries, columns=['human_baseline_summaries', 'original_model_summaries', 'peft_model_summaries'])


In [25]:
rouge = evaluate.load('rouge')

original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)


peft_model_results = rouge.compute(
    predictions=peft_model_summaries,
    references=human_baseline_summaries[0:len(peft_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('PEFT MODEL:')
print(peft_model_results)

ORIGINAL MODEL:
{'rouge1': 0.18856808907609768, 'rouge2': 0.038424229454576256, 'rougeL': 0.1426696244390293, 'rougeLsum': 0.1407115789773711}
PEFT MODEL:
{'rouge1': 0.2529803349988026, 'rouge2': 0.07961374467581733, 'rougeL': 0.17928277865139916, 'rougeLsum': 0.17805671378647303}


In [26]:
import shutil
import os
from google.colab import files

# Define the folder path
folder_path = '/content/peft-Wikisum-summary-checkpoint-local'

# Zip the folder
shutil.make_archive(folder_path, 'zip', folder_path)

# Download the zip file
files.download(folder_path + '.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>